# Recurrent Neural Networks
### Vanilla neural networks and  convolutional neural networks also accept a fixed size vector input and porudce a fixed size vector output
### These models perform this mapping using a fixed amount of computational steps
### RNNs on the other hand allow us to operate over sequences of  vectors - in the inputs, outputs or most generally both
<img src = 'recnn.jpg'/>
<!-- img src = 'house_generate.gif'/-->

### Recurrent nets are a type of artificial neural network designed to recognize patterns in sequences of data, such as text, genomes, handwriting, the spoken word, or numerical times series data emanating from sensors, stock markets and government agencies

### We  have an input  consisting of a sequence of entities  $x^{(1)}\;x^{(2)} \dots \; x^{(r)}\;$ 
### Corresponding to this input we need to either produce a sequence $y^{(1)}\;y^{(2)} \dots y^{(r)}$ or just one output for the entire input sequence y
### We denote the output the RNN produces by $\hat{y}^{(1)}, \hat{y}^{(2)} \dots \hat{y}^{(r)}$
### The case where RNN produces one output for every entity in the single input can be descibed using the following equations:
<h3><center>$h^{(t)} = tanh(Ux^{(t)} + Wh^{(t-1)} + b)$</center></h3>
<h3><center> $\hat{y}^{(t)}=softmax(Vh^{(t)}+c)$</center></h3>
### The RNN computation involves first computing the hideen state for an entity in the sequence denoted by $h^{(t)}$ using the corresponding input at $x^{(t)}$ and the previous hidden state at $h^{(t-1)}$ with weights associated with these denoted by U and W and a bias term b
### There are weights associated with the hidden state denoted by V and a bias term c
### The parameters of the RNN, namely, U,W,V,b,c, etc. are shared across the computation of the hidden layer and output value (for each of the entities in the sequence)
<img src='rnn_eqn_diagram.jpg'/>
## RNN variant with recurrence carried out using output produced in the previous step rather than the previous hidden state
<img src='rnn_prev_output.jpg'/>

# Bidirectional RNNs
### The key intuition behind a bidirectional RNN is to use the entities that lie further in the sequence to make a prediction for the current entity
### It can be described using the following equations:
<h3><center>$h_f^{(t)}=tanh(U_fx^{(t)} + W_f^{(t)} + b_f)$</center></h3>
<h3><center>$h_b^{(t)}=tanh(U_bx^{(t)} + W_b^{(t)} + b_b)$</center></h3>
<h3><center>$\hat{y}^{(t)}=softmax((V_bh_b^{(t)} + V_fh_f^{(t)}+c)$</center></h3>
### The parameters of the RNN, namely, $U_f$ , $U_b$, $W_f$ , $W_b$, $V_b$, $V_f , b_f , b_b$, c, etc. are shared across the computation of the hidden layer and output value (for each of the entities in the sequence). The RNN as described by the equations can process an arbitrarily large input sequence

# Gradients vanishing, explosion and clipping
### During RNN training gradients can drop to a very low value - vanish, shoot up to very large values - exploding
### The technique used is gradient clipping which rescales the norms of the gradients if it shoots up above a user defined threshold which introduces an extra hyperparameter

# LSTM - Long Short Term Memory Networks
### The god of batting is Sachin. If we have to from this sentence predict the last word for the phrase The god of batting is .... - it is fairly straight forward
### Lets consider this set of utterances. I watched Sholay as a kid. It was a fabulous action movie. I took to action movies. Then i went to watch Golmaal. It was an amazing comedy. I feel like watching an old comedy. I will check out again .....   . Here to plug in Golmall we need to go back into the statement and figure it out. We need to remember a hell of a lot more to do this
### LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!
### LSTMs work on the basis of cell states, input gates and forget gates.  For example upon encountering the wish to see an old comedy it can forget information on action movies
### An LSTM can be described with the following set of equations:
<h3><center>$z^{(t)}=g(W_zx^{(t)} + R_z \hat{y}^{(t-1)} + b_z)$</center></h3>
<h3><center>$i^{(t)}=\sigma(W_ix^{(t)} + R_i \hat{y}^{(t-1)} + p_i \otimes c^{(t-1)} + b_i)$</center></h3>
<h3><center>$f^{(t)}=\sigma(W_fx^{(t)} + R_f \hat{y}^{(t-1)} + p_f \otimes c^{(t-1)} + b_f)$</center></h3>
<h3><center>$c^{(t)}=i^{(t)} \otimes z^{(t)} + f^{(t)} \otimes c^{(t-1)}$</center></h3>
<h3><center>$o^{(t)}=\sigma(W_ox^{(t)} + R_o \hat{y}^{(t-1)} + p_o \otimes c^{(t)} + b_o)$</center></h3>
<h3><center>$\hat{y}^{(t)}=o^{(t)}\otimes h(c^{(t)})$</center></h3>
### The most important part is the cell state denoted by $c^{(t)}$
### The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged
### It is updated based on the block input $z^{(t)}$ and the previous cell state $c^{(t-1)}$. The input gate $i^{(t)}$ determines how much of the block makes it into the input the forget gate $f^{(t)}$ how much to the previous cell state to retain
### All the p terms are peephole connections which allow for a fraction of the cell state to factor in the computation of the term in question
<img src='LSTM.JPG'/>

In [6]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open('nietzsche500.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('\nSentences first few elements are {}'.format(sentences[:5]))
print('\nnext_chars first 100 elements are ', next_chars[:100])

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
print('x and y shapes after vectorization are {}, {} respectively'.format(x.shape, y.shape))

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print('x and y first rows after we have populated vectors with 1s for characters respectively are {}, {}'.
     format(x[0, 0, :], y[0, :]))


corpus length: 32124
total chars: 50
nb sequences: 10695

Sentences first few elements are ['preface\n\n\nsupposing that truth is a woma', 'face\n\n\nsupposing that truth is a woman--', 'e\n\n\nsupposing that truth is a woman--wha', '\nsupposing that truth is a woman--what t', 'pposing that truth is a woman--what then']

next_chars first 100 elements are  ['n', 'w', 't', 'h', '?', 's', 'h', 'e', 'o', 'g', 'u', '\n', 'r', 'u', 'e', 'i', ' ', 'a', 'a', ' ', 'i', 's', 'h', 's', 'i', 's', 'f', ' ', ' ', 'e', 'h', 'e', 'e', '\n', 'g', 't', 't', ' ', 'v', 'f', 'l', ' ', ' ', 'd', 's', 'n', 'w', 'e', '-', 'a', 't', ' ', 'r', 'b', '\n', 'r', 'u', 'e', ' ', 'd', 'l', 's', 'i', 'o', 'u', 't', 'w', 'h', 'h', 'h', 'h', ' ', 'v', 'u', 'a', 'y', 'a', '\n', 'e', ' ', 'd', 's', 's', 'o', 'r', 'h', 'h', 'e', 'e', ' ', 's', 'l', 'd', 'n', 'u', 'e', 'l', 'm', 'h', 's']
Vectorization...
x and y shapes after vectorization are (10695, 40, 50), (10695, 50) respectively
x and y first rows after we have popula

In [8]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
print(len)
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model dimensions here will be 4 * ((50 + 1) * 128 + 128 * 128)
# 50 is the size of the input layer, 128 is the size of the output layer
# U will be of dimension (50 + 1) * 128 , W will be of dimension 128 * 128 
# there will be a different set of matrices for the input, forget and output gate
# We will have one more matrix for the cell state 
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model.summary())


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_6 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation_6 (Activation)    (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(500):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
#             print('x_pred after we have switched available characers to 1 is', x_pred[0, 0, :], x_pred[0, 1, :])

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
callbacks=[print_callback])

Epoch 1/60
10695/10695 [==============================] - 14s 1ms/step - loss: 0.5583

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "t europe, furnishes food for thought and"
t europe, furnishes food for thought and to do des ane ventains of which as mose of rus is be the meally to peryable by weach spith on as a out of has reacus of which platoman arropl the wery belomed them thementlysctaince in resertally stisely bely
impersing to bely sthereally the sermanal very for werropt they are the means of have other the caul of the wast mestary and forem. the was ace still ligtl semmalivis philosophers be!ic liscupeat of westive arpition
and to bely work, a care of his perhaps of has pethational senses there mu
----- diversity: 0.5
----- Generating with seed: "t europe, furnishes food for thought and"
t europe, furnishes food for thought and forem.
the wach he stands of sure cermation in as aut the would rerestaves that is to des instrust--at preates

KeyboardInterrupt: 